# Export Weights

If you haven't created a model checkpoint (e.g. model-checkpoint.h5) first go to the Train_Model.ipynb notebook, train a model, and then come back here.

This notebook will export a trained Tensorflow model to JSON weights

### Tensorflow setup

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import os

In [3]:
print("TensorFlow version:", tf.__version__)

gpus = tf.config.experimental.list_physical_devices('GPU')
print("Detected GPUs:", len(gpus))
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

TensorFlow version: 2.9.2
Detected GPUs: 1


### Redefine the model

In [4]:
from tensorflow.keras.initializers import RandomNormal
import tensorflow.keras.backend as K

#Modified depth_to_space shuffle order for easier shader generation
class DepthToSpace2(tf.keras.layers.Layer):
    def __init__(self, input_depth, **kwargs):
        super(DepthToSpace2, self).__init__(**kwargs)
        self.input_depth = input_depth

    def build(self, input_shape):
        super(DepthToSpace2, self).build(input_shape)

    def call(self, x):
        
        x = tf.split(x, (self.input_depth // 4), axis=-1)
        return tf.concat([tf.nn.depth_to_space(xx, 2) for xx in x], axis=-1)


#SR model that doubles image size
def SR2Model(input_texture="MAIN", input_depth=3, highway_depth=4, block_depth=4, init='he_normal', init_last = RandomNormal(mean=0.0, stddev=0.001)):

    input_shape = [None, None, input_depth]
    #Add ".MAIN" in layer name as flag for shader generation, this makes the input act as the MAIN texture
    input_lr = tf.keras.layers.Input(shape=input_shape, name="input." + input_texture)
    input_lr2 = tf.keras.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(input_lr)
    

    x = input_lr
    for i in range(block_depth):
        x = tf.keras.layers.Conv2D(highway_depth, (3, 3), padding='same', kernel_initializer=init)(x)
        x = tf.nn.crelu(x)
    
    x = tf.keras.layers.Conv2D(highway_depth, (3, 3), padding='same', name="conv2d_last",  kernel_initializer=init)(x)
    
    
    #Add "lastresid" in layer name as flag for shader generation, this allows the shader to combine the convolution with the residual add as one layer for faster performance
    #Add ".MAIN" in layer name to make the layer save to the MAIN texture
    x = DepthToSpace2(4, name="depth_to_space2_lastresid." + input_texture)(x)

    
    #Add ".ignore" in layer name as flag for shader generation, this will ignore the layer, as the residual will be added by the previous "lastresid" layer
    x = tf.keras.layers.Add(name="add.ignore." + input_texture)([x, input_lr2])

    model = tf.keras.models.Model(input_lr, x)

    return model

In [5]:
K.reset_uids()
model = SR2Model(input_texture="MAIN", input_depth=3, highway_depth=4, block_depth=3)
model.summary(line_length=150)
model.load_weights("model-checkpoint.h5")

Model: "model"
______________________________________________________________________________________________________________________________________________________
 Layer (type)                                    Output Shape                     Param #           Connected to                                      
 input.MAIN (InputLayer)                         [(None, None, None, 3)]          0                 []                                                
                                                                                                                                                      
 conv2d (Conv2D)                                 (None, None, None, 4)            112               ['input.MAIN[0][0]']                              
                                                                                                                                                      
 tf.compat.v1.nn.crelu (TFOpLambda)              (None, None, None, 8)         

### Generate GLSL files

This code comes directly from the Anime4K repository. We first write weights to `.glsl`files before compiling a `.json` weights file. This is roundabout, but I wrote the parsing script originally to parse the weights from the `.glsl` files, and also Anime4K doesn't have Tensorflow model weights for it's networks, only the weights from the `.glsl` files.

In [6]:
from shaderutils import gen_shader
gen_shader(model, hook="MAIN", file="cnn-2x-custom.glsl", desc="Upscale", when="OUTPUT.w MAIN.w / 1.200 > OUTPUT.h MAIN.h / 1.200 > *")

### Convert GLSL to JSON
This script convers the GLSL file to JSON

First, we have a layer class as a utility file which does most of the parsing

In [ ]:
class Layer:

    def __init__(self, raw_text):
        [descriptor, code] = raw_text.split('//!WHEN OUTPUT.w MAIN.w / 1.200 > OUTPUT.h MAIN.h / 1.200 > *\n')

        self.parse_fields(descriptor)

        if(self.type == "conv"):
            self.parse_code(code)



    def parse_code(self, code):
        print("Name: {}, Input: {}, Outputs: {}".format(self.name, self.inputs, self.output))

        main_fn = code.split('vec4 hook() {\n')[1].split('\n    return result;')[0]

        lines = main_fn.split('\n')

        convolutions = lines[:-1]
        bias = lines[-1]
        
        print(len(convolutions))
        assert (len(convolutions)%9 == 0)
        assert (int(len(convolutions)/9) == len(self.inputs) or int(len(convolutions)/9) == len(self.inputs)*2)
        self.double_input = int(len(convolutions)/9) == len(self.inputs)*2

        self.weights = []

        for i, line in enumerate(convolutions):

            weights = line.split('mat4(')[1].split(')')[0].split(',')
            assert (len(weights) == 16)

            for w in weights:
                self.weights.append(float(w))


        self.bias = [float(b) for b in bias.split('vec4(')[1].split(')')[0].split(',')]




    def parse_outputs(self, fields):

        for field in fields:
            parts = field.split(' ')
            field_name = parts[0][3:]
            field_value = parts[1]



            if(field_name == "SAVE"):
                if(field_value== "MAIN"):
                    self.name = "pixel_shuffle"
                    self.type = "pixel_shuffle"
                    self.output = "canvas"
                else:
                    self.name = field_value
                    self.type = "conv"
                    self.output = field_value


    def parse_inputs(self, fields):


        for field in fields:
            parts = field.split(' ')
            field_name = parts[0][3:]
            field_value = parts[1]

            if(field_name == "BIND"):
                if(field_value == "MAIN"):
                    if(self.output != "canvas"):
                        self.inputs.append("input_image")

                else:
                    self.inputs.append(field_value)


    def parse_fields(self, descriptor):

        fields = descriptor.split("\n")[1:-1]


        self.inputs = []
        self.parse_outputs(fields)
        self.parse_inputs(fields)

Next we define the process file. Adjust the output filename as needed

In [ ]:
import json

def process_file(fname):

    f = open(fname)
    text = f.read()

    f.close()


    raw_layers = text.split('!DESC')[1:]

    layers = [Layer(raw_layer) for raw_layer in raw_layers]

    network_dict = {
        'name': 'anime4k/cnn-2x-s',
        'layers': {}
    }

    for layer in layers:
        layer_dict = {
            'name': layer.name,
            'type': layer.type,
            'inputs': layer.inputs,
            'output': layer.output
        }

        if(layer.type == 'conv'):

            layer_dict['weights'] = layer.weights

            layer_dict['bias'] = layer.bias

        network_dict['layers'][layer.name] = layer_dict

    f = open('{}-custom.json'.format(network_dict['name'].replace('/', '-')), 'w')
    json.dump(network_dict, f)
    f.close()

Run the conversion

In [ ]:
process_file('cnn-2x-custom.glsl')